In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import * 
from other_repos.pyDMS.pyDMS.pyDMS import *
import time

os.environ["GDAL_MAX_DATASET_POOL_SIZE"] = "600"
# get tiles for Brandenburg
# bran = pd.read_csv('/data/Aldhani/eoagritwin/misc/state_tile_csv/clipped_grid_bran_tiles.csv')

# paths
lowmask_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/THUENEN_GER_LST_WARP.tif'
temp_dump = '/data/Aldhani/eoagritwin/et/Sentinel3//LST/LST_values/tempDump/'

# create vrts of slope, aspect and landcover (for masking)
path_to_slope = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/SLOPE/'
path_to_aspect = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/ASPECT/'
path_to_agro = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/THUENEN_2021/'

# get all highRes datasets: S2 composites, aspect, ratio and incedence for the tile/time and stack them (in a composite if more than one tile is processed)
# set the highRes S2 tiles that will be processed
# tiles_to_process = createFORCEtileLIST(list(bran['Tile_X']),
#                                         list(bran['Tile_Y']))

Tile_X = [63,64,63,64]
Tile_Y = [41,41,42,42]

tiles_to_process = createFORCEtileLIST(Tile_X, Tile_Y)

# make a specific folder for this run and store the info together
csv_path = f'{temp_dump}folders.csv'
rand_foldname = getUniqueIDfromTILESXY(Tile_X, Tile_Y)
temp_dump_fold = f'{temp_dump}{rand_foldname}/'

if not os.path.exists(temp_dump_fold):
    os.makedirs(temp_dump_fold,exist_ok=False)
    df = pd.DataFrame({
            'Folder': temp_dump_fold,
            'Tile_X': Tile_X,
            'Tile_Y': Tile_Y   
        })

    if not os.path.exists(csv_path):
        df.to_csv(csv_path, index=False)
    else:
        df_exist = pd.read_csv(csv_path)
        df_new = pd.concat([df_exist, df], ignore_index=True)
        df_new.to_csv(csv_path, index=False)
        # add the lines

    # slope-tiles
    slopes = [file for file in getFilelist(path_to_slope, '.tif') if any(tile in file for tile in tiles_to_process)] # if any tile name is in file
    # aspect-tiles
    aspects = [file for file in getFilelist(path_to_aspect, '.tif') if any(tile in file for tile in tiles_to_process)] # if any tile name is in file
    # thuenen-tiles
    thuenen = [file for file in getFilelist(path_to_agro, '.tif') if any(tile in file for tile in tiles_to_process)] # if any tile name is in file

    # get those tiles (and composite if more than one tile is provided)
    if len(tiles_to_process) == 1:
        
        slope_path = slopes[0]
        aspect_path = aspects[0]
        thuenen_path = thuenen[0]

    else:
        slope_path = f'{temp_dump_fold}SLOPE_vrt'
        gdal.BuildVRT(slope_path, slopes)

        aspect_path = f'{temp_dump_fold}ASPECT.vrt'
        gdal.BuildVRT(aspect_path, aspects)

        thuenen_path = f'{temp_dump_fold}THUENEN.vrt'
        gdal.BuildVRT(thuenen_path, thuenen)
else:
    raise ValueError(f'{temp_dump_fold} already exists!!!!')

/data/Aldhani/users/potzschf/conda/envs/workhorse/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [2]:
# year 
year = 2019#range(2017,2025):
  
# paths
path_to_S2_tiles = f'/data/Aldhani/eoagritwin/force/output/Guzinski/{year}/'

##### which tiles should be processed
# get a list with all available tiles
files = getFilelist(f'{path_to_S2_tiles}/tiles', '.tif', deep=True) 
files = [file for file in files if any(tile in file for tile in tiles_to_process)]
date_list = check_forceTSI_compositionDates(files)

th_ds = gdal.Open(thuenen_path)
th_arr = th_ds.GetRasterBand(1).ReadAsArray()
mask = np.where(th_arr == -9999, 0, 1)

lowRes_files = []
highRes_files = []
highRes_names = []
#### S2 composites are time sensitive (need to be aligned with date of LST observation), so is incidence
for date in date_list:

    # if not os.path.exists(f'{temp_dump_fold}INCIDENCE_{date}.vrt'):
    if date != '20190705':
        continue
    # get those tiles (and composite if more than one tile is provided)
    if len(tiles_to_process) == 1:

        tilesS2 = [file for file in getFilelist(path_to_S2_tiles, '.tif', deep=True) if tiles_to_process[0] in file and f'{date}.tif' in file]
        S2_path = f'{temp_dump_fold}S2_{date}.vrt'
        gdal.BuildVRT(S2_path, tilesS2)

    else:
        tilesS2 = [file for file in getFilelist(path_to_S2_tiles, '.tif', deep=True) if any(tile in file for tile in tiles_to_process) and f'{date}.tif' in file] 
        force_to_vrt(tilesS2,
                getCOLORSinOrderFORCELIST(tilesS2, list(dict.fromkeys(tile.split('SEN2L_')[-1].split('_TSI')[0] for tile in tilesS2)), single=False),
                f'{temp_dump_fold}S2_{date}',
                False,
                bandnames= list(dict.fromkeys(tile.split('SEN2L_')[-1].split('_TSI')[0] for tile in tilesS2)))
        print(list(dict.fromkeys(tile.split('SEN2L_')[-1].split('_TSI')[0] for tile in tilesS2)))
        S2_path = [file for file in getFilelist(f'{temp_dump_fold}S2_{date}', '.vrt', deep=True) if '_Cube' in file][0]
        
        # determine LST and incidence files associated with respective S2 composite
    band_dict = transform_compositeDate_into_LSTbands(date, 4)


    # stat used for compositing
    for comp_stat in ['minVZA', 'maxLST']:
        path_to_incident = f'/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/INCIDENCE/{comp_stat}/{year}/'
        path_to_LST = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/{comp_stat}/{year}/'

        # get all LST bands that can be sharped with the S2 composite at this date (and sun angle incidence files as well, as they are dependent on that date
        LSTs = []

        for k, v in band_dict.items():
            month = v['month']
            band = int(v['band'])
            v_path = f'{path_to_LST}Daily_LST_{comp_stat}_{year}_{month}.tif'
            ds = gdal.Open(v_path, 0)
            LST_arr = ds.GetRasterBand(band).ReadAsArray() # store as single Tiff in temp
            makeTif_np_to_matching_tif(LST_arr, v_path, f'{temp_dump_fold}Daily_LST_{comp_stat}_{year}_{month}_{band:02d}.tif')

            # store the paths for selecting incidence for corresponding LST
            incid_date = f'{year}_{month}_{band:02d}.tif'
            lowRes_files.append(f'{temp_dump_fold}Daily_LST_{comp_stat}_{year}_{month}_{band:02d}.tif')

            # incidence-tiles
            incids = [file for file in getFilelist(path_to_incident, '.tif', deep=True) if any(tile in file for tile in tiles_to_process) and incid_date in file] 
            # get those tiles (and composite if more than one tile is provided)
            if len(tiles_to_process) == 1:
                incid_path = incids[0]

            else:
                incid_path = f'{temp_dump_fold}INCIDENCE_{comp_stat}_{incid_date.split('.')[0]}.vrt'
                gdal.BuildVRT(incid_path, incids)

            # create highRes file through exapnding the vrt of S2
            highRes_path = f'{temp_dump_fold}HIGHRES_{comp_stat}_{incid_date.split('.')[0]}.vrt'
            gdal.BuildVRT(highRes_path, [S2_path, slope_path, aspect_path, incid_path], separate=True)
            highRes_files.append(highRes_path)
            highRes_names.append('S2notMasked')
            maskVRT(highRes_path, mask)
            highRes_files.append(f'{temp_dump_fold}HIGHRES_{comp_stat}_{incid_date}')
            lowRes_files.append(f'{temp_dump_fold}Daily_LST_{comp_stat}_{year}_{month}_{band:02d}.tif')
            highRes_names.append('S2Masked')

all dates of composites are the same :)
/data/Aldhani/eoagritwin/et/Sentinel3//LST/LST_values/tempDump/2514b73c638b86ba52047c6c65eea221652398b499d492380209a557c938f1ba/S2_20190705/Force_X_from_63_to_64_Y_from_41_to_42/
/data/Aldhani/eoagritwin/et/Sentinel3//LST/LST_values/tempDump/2514b73c638b86ba52047c6c65eea221652398b499d492380209a557c938f1ba/S2_20190705/Force_X_from_63_to_64_Y_from_41_to_42/Force_X_from_63_to_64_Y_from_41_to_42_0.vrt
/data/Aldhani/eoagritwin/et/Sentinel3//LST/LST_values/tempDump/2514b73c638b86ba52047c6c65eea221652398b499d492380209a557c938f1ba/S2_20190705/Force_X_from_63_to_64_Y_from_41_to_42/Force_X_from_63_to_64_Y_from_41_to_42_1.vrt
/data/Aldhani/eoagritwin/et/Sentinel3//LST/LST_values/tempDump/2514b73c638b86ba52047c6c65eea221652398b499d492380209a557c938f1ba/S2_20190705/Force_X_from_63_to_64_Y_from_41_to_42/Force_X_from_63_to_64_Y_from_41_to_42_2.vrt
/data/Aldhani/eoagritwin/et/Sentinel3//LST/LST_values/tempDump/2514b73c638b86ba52047c6c65eea221652398b499d492380209

In [6]:
joblist = []
outFolder = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/{rand_foldname}/'
for idx, highResFilename in enumerate(highRes_files):
    lowResFilename = lowRes_files[idx]
    f1 = f'{outFolder}{'/'.join(highResFilename.split('.')[0].split('_')[2:6])}/'
    for maskname, mask_lowRes in zip(['withoutLSTmask', 'withLSTmask'], ['', lowmask_path]):
        lowResMaskFilename = mask_lowRes
        f2 = f'{f1}{maskname}/'
        for movWin in range(15,60,10):
            for cv in range(5,55,10):
                for regrat in [0.1, 0.2, 0.3]:
                    kombi = f'mvwin{movWin}_cv{cv}_regrat{int(regrat*100):02d}_{highRes_names[idx]}_{maskname}'
                    f3 = f'{f2}{highRes_names[idx]}/'
                    os.makedirs(f3, exist_ok=True)
                    os.makedirs(f'{f3}residuals/', exist_ok=True)
                    joblist.append([highResFilename, 
                                lowResFilename,
                                mask_lowRes,
                                cv, movWin, regrat,
                                f'{f3}{'_'.join(highResFilename.split('.')[0].split('_')[2:6])}_{kombi}.tif'])

In [8]:
outputFilename = joblist[0][-1]

In [11]:
print(f'{os.path.split(outputFilename)[0]}/Values/{os.path.split(outputFilename)[1]}')
print(f'{os.path.split(outputFilename)[0]}/residuals/{os.path.split(outputFilename)[1]}_residual{os.path.splitext(outputFilename)[1]}')

/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/2514b73c638b86ba52047c6c65eea221652398b499d492380209a557c938f1ba/minVZA/2019/July/01/withoutLSTmask/S2notMasked/Values/minVZA_2019_July_01_mvwin15_cv5_regrat10_S2notMasked_withoutLSTmask.tif
/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/2514b73c638b86ba52047c6c65eea221652398b499d492380209a557c938f1ba/minVZA/2019/July/01/withoutLSTmask/S2notMasked/residuals/minVZA_2019_July_01_mvwin15_cv5_regrat10_S2notMasked_withoutLSTmask.tif_residual.tif


In [ ]:
    for idx, LST_file in enumerate(LSTs):
            # incidence-tiles
            incids = [file for file in getFilelist(path_to_incident, '.tif', deep=True) if any(tile in file for tile in tiles_to_process) and incid_dates[idx] in file] 
            # get those tiles (and composite if more than one tile is provided)
            if len(tiles_to_process) == 1:
                incid_path = incids[0]

            else:
                incid_path = f'{temp_dump_fold}INCIDENCE_{comp_stat}_{incid_dates[idx]}.vrt'
                gdal.BuildVRT(incid_path, incids)

            # sanity check for incidence and LST date
            if (LSTs[idx].split(f'{year}')[-1] == incids[0].split(f'{year}')[-1]):
                
                # get LST file
                lowRes_files.append(LSTs[idx])
                # create highRes file through exapnding the vrt of S2
                highRes_path = f'{temp_dump_fold}HIGHRES_{comp_stat}_{incid_dates[idx]}.vrt'
                gdal.BuildVRT(highRes_path, [S2_path, slope_path, aspect_path, incid_path], separate=True)
                maskVRT(highRes_path, mask)
                highRes_files.append(f'{temp_dump_fold}HIGHRES_{comp_stat}_{incid_dates[idx]}.tif')

            else:
                raise ValueError('Something is seriously wrong with the alignment of LST and incidence dates!!!!!')

In [ ]:
'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/2514b73c638b86ba52047c6c65eea221652398b499d492380209a557c938f1ba/'

In [63]:
files = getFilelist('/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/2514b73c638b86ba52047c6c65eea221652398b499d492380209a557c938f1ba/', '.tif') #temp_dump_fold

In [66]:
fnames = [file.split('/')[-1] for file in files if not 'residual' in file]

In [ ]:
for file in fnames:
    if file.split('_')[0]